In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from tqdm import tqdm_notebook as tqdm

import pickle
import os

In [2]:
img_size=(256,256)

In [3]:
def calculateRadius(points,thresh=0.7):
    
    alpha=points[2]-points[0]
    beta=points[3]-points[1]
    

    disc=alpha*beta*(1-thresh)
    disc/=alpha+beta
    disc=np.sqrt(disc)
    disc=np.floor(disc)
    disc=disc.astype(np.int32)
    
    return disc

In [4]:
def create_mask(img_size,points,mode='top_left',varience=1.5,normalized=True):
    """
    return normalized mask uint8 range 255
    shape [img_size[0],img_size[1]]
    """
    mask=np.zeros((img_size[0],img_size[1]))
    radius=calculateRadius(points)
    sigma=varience*radius
    sigma*=sigma
    sigma=2*sigma
    if mode=='top_left':
        
        cv2.circle(mask,(int(points[0]),int(points[1])),radius, (1), -1)
       
        s=np.array(np.meshgrid(np.arange(img_size[0]),np.arange(img_size[0]))).transpose([1,2,0]).reshape(-1,2).astype(np.float32)
        origin=np.floor(np.array([points[0],points[1]]))
    
    elif mode=='bottom_right':
        
        cv2.circle(mask,(int(points[2]),int(points[3])),radius, (1), -1)
        s=np.array(np.meshgrid(np.arange(img_size[0]),np.arange(img_size[0]))).transpose([1,2,0]).reshape(-1,2).astype(np.float32)
        origin=np.floor(np.array([points[2],points[3]]))
    else:
        print("Mode error")
        return
        
    s-=origin
    s=np.square(s)
    s=np.sum(s,axis=1)
    s/=sigma
    s=np.exp(-s)
    s=s.reshape(img_size)
#     s=s/np.max(s)
    if not normalized:
        s=s*255
    
    return s

In [5]:

data=pd.read_csv('training_set.csv')
data=data.set_index('image_name')
data=data[['x1','y1','x2','y2']]
data['x1']=data['x1']*img_size[0]/640
data['x2']=data['x2']*img_size[0]/640
data['y1']=data['y1']*img_size[1]/480
data['y2']=data['y2']*img_size[1]/480

In [6]:
data.head()

,x1,y1,x2,y2
image_name,,,,
JPEG_20160517_140621_1000651031832.png,31.2,44.266667,178.4,225.066667
1473231510890DeeplearnS11456.png,29.2,77.866667,226.8,187.200000
1476881557883Untitled1.png,5.2,66.133333,224.4,193.066667
1474264827890cobalt-beyond-the-sea-maxi-dress3.png,0.0,74.666667,256.0,176.000000
JPEG_20160711_113321_1000235741992.png,86.8,34.666667,176.8,248.000000


In [7]:
filenames=[]
# f=open('train_list.txt','w')
for x,_ in data.iterrows():
    filenames.append(x)
#     f.write(x)
#     f.write('\n')
# f.close()
print(len(filenames))

24000


In [8]:
label_dict={}
for file in filenames:
    points=np.array(data.loc[file])
    points=np.floor(points)
    label_dict[file]=points
print("completed")

with open('labels_xy.pickle','wb') as handle:
    pickle.dump(label_dict,handle,protocol=pickle.HIGHEST_PROTOCOL)
print("saved")

completed
saved


In [9]:
left_path='../labels/top'
right_path='../labels/bottom'
with open('labels_xy.pickle', 'rb') as handle:
    labels_xy = pickle.load(handle)

In [10]:

for file in tqdm(filenames):
    try:
        lsv=os.path.join(left_path,file)
        rsv=os.path.join(right_path,file)
        points=labels_xy[file]
        varience=1.75
        left_mask=create_mask(img_size,points,mode='top_left',varience=varience,normalized=False)
        right_mask=create_mask(img_size,points,mode='bottom_right',varience=varience,normalized=False)
        cv2.imwrite(lsv,left_mask)
        cv2.imwrite(rsv,right_mask)
    except:
        print("Error in {} ".format(file))
print("Completed")

/home/aghinsa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


Error in 1471600982974DSC_0322.png 
Error in 1477035945016DSC_7534.png 

Completed


Remove images causing errors from train_list


Error in 1471600982974DSC_0322.png  
Error in 1477035945016DSC_7534.png